# Homework 1 Scratch

Not a solution to the homework problems, but a description of some work for feedback.

## Problem Statement

Use a Chebyshev method to solve the second order ordinary differential equation

$$ u''(t) + a u'(t) + b u(t) = f(t) $$

from $t=0$ to $t=1$ with initial conditions $u(0) = 1$ and $u'(0) = 0$.

* Do a grid convergence study to test the accuracy of your method.
* Setting $f(x)=0$, experiment with the values $a$ and $b$ to identify two regimes with qualitatively different dynamics.

## Approach

As an intial code base, I copied the general Chebyshev functions from the FDHighOrder notebook and made a few changes to the `laplacian_cheb` function to fit the homework ODE.

In [1]:
%precision 3
%matplotlib notebook

import numpy
from matplotlib import pyplot
pyplot.style.use('ggplot')

In [2]:
# The code in this cell was taken from the class notebook FDHighOrder

def vander_chebyshev(x, n=None):
    """
    Generates a Vandermond matrix using Chebyshev polynomials.
    """
    if n is None:
        n = len(x)
    T = numpy.ones((len(x), n))
    if n > 1:
        T[:,1] = x
    for k in range(2,n):
        T[:,k] = 2 * x * T[:,k-1] - T[:,k-2]
    return T


def chebeval(z, n=None):
    """
    Build matrices to evaluate the n-term Chebyshev expansion and its derivatives at point(s) z
    
    Returns three matrices:
      Tz: Vandermonde matrix evaluating the n-term Chebychev expansion at the points in z
      dTz: first derivative of Tz
      ddTz: second derivative of Tz
    """
    z = numpy.array(z, ndmin=1)
    if n is None:
        n = len(z)
    Tz = vander_chebyshev(z, n)
    dTz = numpy.zeros_like(Tz)
    dTz[:,1] = 1
    dTz[:,2] = 4*z
    ddTz = numpy.zeros_like(Tz)
    ddTz[:,2] = 4
    for n in range(3,n):
        dTz[:,n]  = n * (2*Tz[:,n-1] + dTz[:,n-2]/(n-2))
        ddTz[:,n] = n * (2*dTz[:,n-1] + ddTz[:,n-2]/(n-2))
    return [Tz, dTz, ddTz]

def cosspace(a, b, n=50):
    return (a + b)/2 + (b - a)/2 * (numpy.cos(numpy.linspace(-numpy.pi, 0, n)))

def hw_cheb(n, rhsfunc, left, right, a = 1, b = 1):
    """
    Solve the homework 1 initial value problem on (0, 1) using n elements with rhsfunc(x) forcing.
    The left and right boundary conditions are specified as a pair (deriv, func) where
      * deriv=0 for Dirichlet u(x_endpoint) = func(x_endpoint)
      * deriv=1 for Neumann u'(x_endpoint) = func(x_endpoint)
    """
    x = cosspace(-1, 1, n+1)  # n+1 points is n "elements"
    
    # T[0]: Chebychev expansion evaluated at points in x
    # T[1]: first derivative of Chebychev expansion evaluated at points in x
    # T[2]: second derivative of Chebychev expansion evaluated at points in x
    T = chebeval(x)
    
    # This is our left hand side, u''(t) + au'(t) + bu(t)
    L = T[2] + a * T[1] + b * T[0]
    
    # Our right hand side is the forcing function f(t)
    rhs = rhsfunc(x)
    
    # Now we'll apply the boundary conditions with the first and last rows (0 and -1) of the operator
    # `left` and `right` are pairs of the derivative number and the condition on it (see docstring)
    for i,deriv,func in [(0, *left), (-1, *right)]:        
        # ith row of operator L is the ith row of the derivth derivative
        L[i] = T[deriv][i]
        
        rhs[i] = func(x[i])
    return x, L.dot(numpy.linalg.inv(T[0])), rhs

class exact_tanh:
    def __init__(self, k=1, x0=0):
        self.k = k
        self.x0 = x0
    def u(self, x):
        return numpy.tanh(self.k*(x - self.x0))
    def du(self, x):
        return self.k * numpy.cosh(self.k*(x - self.x0))**(-2)
    def ddu(self, x):
        return -2 * self.k**2 * numpy.tanh(self.k*(x - self.x0)) * numpy.cosh(self.k*(x - self.x0))**(-2)


Changing course to just looking at the problem--

$$ u''(t) + a u'(t) + b u(t) = f(t)\\
0 \leq t \leq 1\\
u(0) = 1\\
u'(0) = 0$$

To convert this second-order ODE to a first-order system, add variables $u_0=u$, $u_1=u_0'$. We end up with

$$ u_1' + au_1 + bu_0 = f(t)$$

Rearranging it--

$$\begin{align*}
u_0' &= u_1\\
u_1' &= f(t) - au_1 - bu_0 
\end{align*}$$

Considering a matrix, 

$$\begin{align*}
\mathbf{u} &= \begin{bmatrix}
 u_0\\
 u_1
\end{bmatrix}\\
\mathbf{u}' &= A(t)\mathbf{u} + \mathrm{source}(t)
\end{align*}$$

Written out,

$$\begin{align*}
\mathbf{u}' &= 
\begin{bmatrix}
 0 & 1\\
-b & -a
\end{bmatrix}\mathbf{u} + 
\begin{bmatrix}
 0\\
 f(t)
\end{bmatrix}
\end{align*}$$

### Another Pass

Still convert the second-order ODE to a first-order system by adding variables $u_0=u$, $u_1=u_0'$:

$$ u_1' + au_1 + bu_0 = f(t)$$

Our system is

$$ u_0' - u_1 = 0\\
u_1' + au_1 + bu_0 = f$$

Consider this system using matrices:

$$\begin{bmatrix}
 \frac{\mathrm{d} }{\mathrm{d} t} & -1\\
  b & \frac{\mathrm{d} }{\mathrm{d} t} + a
\end{bmatrix}
\begin{bmatrix}
 u_0\\
 u_1
\end{bmatrix} = 
\begin{bmatrix}
 0\\
 f
\end{bmatrix}$$

On the way to discretizing this, start by considering $u_0$, $u_1$, $0$, and $f$ as vectors of length $n$. So, for example, the vector for $f$ will contain values of $f$ evaluated at the $n$ points).